In [1]:
from gpt2 import DataLoaderLite, TrainGPT

In [2]:
# Load Dataloader
train_loader = DataLoaderLite(B=8, T=1024, file='input.txt')
trainer = TrainGPT(model_compile=False)


loaded 338025 tokens
1 epoch = 41 batches
using device cuda


In [3]:
trainer.train_gpt(train_loader, 100000, 100)

c:\repos\ERA_V2\S21-Assignment\gpt2.py:65: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  y = F.scaled_dot_product_attention(q, k, v, is_causal=True)


Step 0, loss: 11.030921936035156, dt: 428.05ms tok/sec: 19137.86
Step 100, loss: 5.84446907043457, dt: 157.00ms tok/sec: 52177.69
Step 200, loss: 5.322109222412109, dt: 166.00ms tok/sec: 49348.23
Step 300, loss: 5.2536139488220215, dt: 161.00ms tok/sec: 50881.98
Step 400, loss: 4.880702018737793, dt: 161.62ms tok/sec: 50685.78
Step 500, loss: 4.45494270324707, dt: 164.12ms tok/sec: 49915.29
Step 600, loss: 4.921161651611328, dt: 162.00ms tok/sec: 50567.99
Step 700, loss: 4.42236852645874, dt: 161.00ms tok/sec: 50882.66
Step 800, loss: 4.441651821136475, dt: 160.00ms tok/sec: 51200.05
Step 900, loss: 4.115748405456543, dt: 161.00ms tok/sec: 50882.28
Step 1000, loss: 4.408835411071777, dt: 161.51ms tok/sec: 50722.59
Step 1100, loss: 4.0244550704956055, dt: 162.00ms tok/sec: 50567.85
Step 1200, loss: 4.29283332824707, dt: 161.12ms tok/sec: 50843.06
Step 1300, loss: 4.194064140319824, dt: 161.00ms tok/sec: 50881.23
Step 1400, loss: 4.005340576171875, dt: 162.51ms tok/sec: 50409.67
Step 150

KeyboardInterrupt: 

In [6]:
import tiktoken
import torch
from torch.nn import functional as F


In [18]:
num_return_sequences = 5
max_length = 100

device = trainer.device
model.to(device)

enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode("Hello world")
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences,1)
x = tokens.to(device)




torch.manual_seed(42)
torch.cuda.manual_seed(42)


model = trainer.model

while x.size(1) < max_length:
    with torch.no_grad():
        # get logits from the model
        logits, loss = model(x)
        logits = logits[:,-1,:]
        probs = F.softmax(logits, dim=-1)

        topk_prob, topk_indices = torch.topk(probs, 50, dim=-1)

        ix = torch.multinomial(topk_prob, 1)

        xcol = torch.gather(topk_indices, -1, ix)

        x = torch.cat((x,xcol), dim=1)

# print the generated text
for i in range(num_return_sequences):
    tokens = x[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(f"Sentence {i+1}","-"*30)
    print( decoded)

Sentence 1 ------------------------------
Hello world: tell toant' heavenly his have strengthen
Of any, there please you might help
Byces

Against this, they do liege.
If this, at danger at least.
To As not
By heaven, not helpings,

' me.
If our brotherhood
Of Buckingham, bid us.


Of your queen:
If this, to speak,
Set us

To what thou declare
The issue on,

Sentence 2 ------------------------------
Hello world: tell toant and duke with too areWill any man to quoth this, bid
By nothing
E: if it is meetonged trate colours my crown.

 from go from home; we been

He crown.


In love these two, andFor shame forgetly mine own life, shall go for their own is left toward your experience and 'twere duty were from Time, as the king?
Art thou hast crown.
Sentence 3 ------------------------------
Hello world: tell to, in word
Of thatof o'ld Pompey enough,
By ever think there dies and fortyen his careful! what thou hast, still rogue, till ever thou hast thou hast thou hast disposed


Or spourn for

In [15]:
model.to('cpu')

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50304, 768)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50304, bias=False)
)

In [16]:
torch.save(model, 'GPT_124M.pth')

In [19]:
from torchsummary import summary

summary(model,x)

Layer (type:depth-idx)                             Output Shape              Param #
├─ModuleDict: 1                                    []                        --
|    └─Embedding: 2-1                              [-1, 768]                 786,432
|    └─Embedding: 2-2                              [-1, 100, 768]            38,633,472
|    └─ModuleList: 2                               []                        --
|    |    └─Block: 3-1                             [-1, 100, 768]            7,087,872
|    |    └─Block: 3-2                             [-1, 100, 768]            7,087,872
|    |    └─Block: 3-3                             [-1, 100, 768]            7,087,872
|    |    └─Block: 3-4                             [-1, 100, 768]            7,087,872
|    |    └─Block: 3-5                             [-1, 100, 768]            7,087,872
|    |    └─Block: 3-6                             [-1, 100, 768]            7,087,872
|    |    └─Block: 3-7                             [-1, 100,

Layer (type:depth-idx)                             Output Shape              Param #
├─ModuleDict: 1                                    []                        --
|    └─Embedding: 2-1                              [-1, 768]                 786,432
|    └─Embedding: 2-2                              [-1, 100, 768]            38,633,472
|    └─ModuleList: 2                               []                        --
|    |    └─Block: 3-1                             [-1, 100, 768]            7,087,872
|    |    └─Block: 3-2                             [-1, 100, 768]            7,087,872
|    |    └─Block: 3-3                             [-1, 100, 768]            7,087,872
|    |    └─Block: 3-4                             [-1, 100, 768]            7,087,872
|    |    └─Block: 3-5                             [-1, 100, 768]            7,087,872
|    |    └─Block: 3-6                             [-1, 100, 768]            7,087,872
|    |    └─Block: 3-7                             [-1, 100,